<a href="https://colab.research.google.com/github/vatsasree/CV-Assignment-2/blob/main/CV_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision.datasets
from torch.utils.data import DataLoader
import cv2
import numpy as np


In [ ]:
train_set = torchvision.datasets.MNIST(
    root='./data', train=True, download=True,
    transform=torchvision.transforms.ToTensor()
    )

test_set = torchvision.datasets.MNIST(
    root='./data', train=False, download=True,
    transform=torchvision.transforms.ToTensor()
    )


100%|██████████| 9912422/9912422 [00:00<00:00, 89479406.09it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 35326828.18it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 27456434.25it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 6416479.88it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
print(train_set)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [ ]:
print(test_set)

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [ ]:
batch_size = 60
n_workers = 4

In [ ]:
train_dataset = DataLoader(train_set,
                           shuffle=True)

In [ ]:
test_dataset = DataLoader(test_set,
                           shuffle=True)

In [ ]:
# def extract_sift_features(image):
#     # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     if image.dtype != np.uint8:
#         image = (image * 255).astype(np.uint8)
#     gray = image
#     print(gray.shape, type(gray))
#     sift = cv2.SIFT_create()
#     keypoints, descriptors = sift.detectAndCompute(gray, None)
#     return keypoints, descriptors

# from sklearn.cluster import KMeans

# def cluster_sift_features(descriptors, num_clusters):
#     kmeans = KMeans(n_clusters=num_clusters)
#     kmeans.fit(descriptors)
#     return kmeans.cluster_centers_

# def compute_histogram(keypoints, descriptors, codebook):
#     # kmeans = KMeans(n_clusters=codebook.shape[0], init=codebook)
#     kmeans = KMeans(n_clusters=num_clusters)
#     kmeans.fit(descriptors)
#     histogram = np.zeros(codebook.shape[0])
#     for cluster_label in kmeans.labels_:
#         histogram[cluster_label] += 1
#     return histogram

def compute_cluster_centers(descriptors, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(descriptors)
    return kmeans.cluster_centers_

# Function to compute histogram of visual words for an image
def compute_histogram(descriptors, cluster_centers):
    histogram = np.zeros(len(cluster_centers))
    for descriptor in descriptors:
        distances = np.linalg.norm(cluster_centers - descriptor, axis=1)
        nearest_cluster = np.argmin(distances)
        histogram[nearest_cluster] += 1
    return histogram


In [ ]:
# from sklearn import svm
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Extract SIFT features, compute histograms, and build dataset
# X = []
# y = []

# num_clusters = 100


# for image, labels in train_dataset:
#     print(image.shape)
#     label = labels.item()
#     image = image[0].permute(1, 2, 0).numpy()
#     print(image.shape, type(image))
#     # gray_image = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_BGR2GRAY)
#     # gray_image = image.astype(np.uint8)
#     # gray_image = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_GRAY2BGR)
#     keypoints, descriptors = extract_sift_features(image)
#     print(len(keypoints), len(descriptors))
#     codebook = cluster_sift_features(descriptors, num_clusters)
#     histogram = compute_histogram(keypoints, descriptors, codebook)
#     X.append(histogram)
#     y.append(label)

# X = np.array(X)
# y = np.array(y)

# # Split dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train linear SVM model
# svm_model = svm.LinearSVC()
# svm_model.fit(X_train, y_train)

# # Predict on the test set
# y_pred = svm_model.predict(X_test)

# # Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)


In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml

# Function to compute cluster centers using K-means
def compute_cluster_centers(descriptors, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(descriptors)
    return kmeans.cluster_centers_

# Function to compute histogram of visual words for an image
def compute_histogram(descriptors, cluster_centers):
    histogram = np.zeros(len(cluster_centers))
    for descriptor in descriptors:
        distances = np.linalg.norm(cluster_centers - descriptor, axis=1)
        nearest_cluster = np.argmin(distances)
        histogram[nearest_cluster] += 1
    return histogram

# # Load MNIST dataset
# mnist = fetch_openml('mnist_784', version=1)
# X, y = mnist.data, mnist.target.astype(int)

def full(num_clusters):
    # Initialize SIFT
    X_train, y_train = train_set.data.numpy(), train_set.targets.numpy()

    sift = cv2.SIFT_create()

    # Initialize lists to store descriptors
    descriptors_list = []
    labels_list = []

    for i in range(len(X_train)):
        image = X_train[i]
        keypoints, descriptors = sift.detectAndCompute(image, None)
        # descriptors_list.append(descriptors)
        if descriptors is not None and descriptors.any():
            descriptors_list.append(descriptors)
            labels_list.append(y_train[i])  # Append corresponding label

    # descriptors_list = [descriptors for descriptors in descriptors_list if descriptors is not None and descriptors.any()]

    # Convert descriptors to a single numpy array
    if descriptors_list:
        descriptors_array = np.concatenate(descriptors_list, axis=0)

        # Choose the number of clusters for K-means
        num_clusters = num_clusters

        # Compute cluster centers using K-means
        cluster_centers = compute_cluster_centers(descriptors_array, num_clusters)

        # Compute histograms of visual words for all images
        t_histograms = []
        t_labels = []
        for descriptors, label in zip(descriptors_list, labels_list):
            histogram = compute_histogram(descriptors, cluster_centers)
            t_histograms.append(histogram)
            t_labels.append(label)

        # Convert histograms and labels to numpy arrays
        X_train = np.array(t_histograms)
        y_train = np.array(t_labels)

        # Create a Linear SVM model
        svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1.0))

        # Train the SVM model
        svm_model.fit(X_train, y_train)
    else:
        print("No descriptors found. Unable to create SVM model.")

    # Assuming you have a separate test set, similar to the training set
    X_test, y_test = test_set.data.numpy(), test_set.targets.numpy()

    # Initialize lists to store test descriptors and labels
    test_descriptors_list = []
    test_labels_list = []

    for i in range(len(X_test)):
        image = X_test[i]
        keypoints, descriptors = sift.detectAndCompute(image, None)
        if descriptors is not None and descriptors.any():
            test_descriptors_list.append(descriptors)
            test_labels_list.append(y_test[i])

    # Compute histograms of visual words for all test images
    test_histograms = []
    test_labels = []
    for descriptors, label in zip(test_descriptors_list, test_labels_list):
        histogram = compute_histogram(descriptors, cluster_centers)
        test_histograms.append(histogram)
        test_labels.append(label)

    # Convert test histograms and labels to numpy arrays
    X_test = np.array(test_histograms)
    y_test = np.array(test_labels)

    # Make predictions using the trained SVM model
    y_pred = svm_model.predict(X_test)

    # Calculate the accuracy of the model
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy of the SVM model on the test set:", accuracy)

    import pickle

    filename = 'finalized_model_{}.sav'.format(num_clusters)
    pickle.dump(svm_model, open(filename, 'wb'))


    return accuracy



In [ ]:
clusters=[10,20,40,80,160,320]
accuracies=[]
for num_clusters in clusters:
  accuracy = full(num_clusters)
  accuracies.append(accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy of the SVM model on the test set: 0.44873366013071897


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy of the SVM model on the test set: 0.5313521241830066


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy of the SVM model on the test set: 0.7238562091503268


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy of the SVM model on the test set: 0.7952410130718954


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
# import pickle

# filename = 'finalized_model_2.sav'
# pickle.dump(svm_model, open(filename, 'wb'))



In [ ]:
# # Assuming you have a separate test set, similar to the training set
# X_test, y_test = test_set.data.numpy(), test_set.targets.numpy()

# # Initialize lists to store test descriptors and labels
# test_descriptors_list = []
# test_labels_list = []

# for i in range(len(X_test)):
#     image = X_test[i]
#     keypoints, descriptors = sift.detectAndCompute(image, None)
#     if descriptors is not None and descriptors.any():
#         test_descriptors_list.append(descriptors)
#         test_labels_list.append(y_test[i])

# # Compute histograms of visual words for all test images
# test_histograms = []
# test_labels = []
# for descriptors, label in zip(test_descriptors_list, test_labels_list):
#     histogram = compute_histogram(descriptors, cluster_centers)
#     test_histograms.append(histogram)
#     test_labels.append(label)

# # Convert test histograms and labels to numpy arrays
# X_test = np.array(test_histograms)
# y_test = np.array(test_labels)

# # Make predictions using the trained SVM model
# y_pred = svm_model.predict(X_test)

# # Calculate the accuracy of the model
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy of the SVM model on the test set:", accuracy)


# Part 2

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 11.2 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import wandb
from tqdm import tqdm
import torchvision
import torchvision.transforms as transforms
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 10
batch_size = 64
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform = transforms.Compose([
                                                  transforms.Resize((28,28)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform = transforms.Compose([
                                                  transforms.Resize((28,28)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]))

# Data loaders
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

model_lenet = LeNet()


In [ ]:
class TransformerEncoderModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(TransformerEncoderModel, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.patch_embedding = nn.Conv2d(1, hidden_dim, kernel_size=5, stride=2, padding=3)
        self.pos_embedding = nn.Parameter(torch.randn(1, hidden_dim, 16, 16))
        self.transformer_encoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(hidden_dim, 8, hidden_dim * 4, 0.1), num_layers)
        self.fc = nn.Linear(hidden_dim * 16 * 16, output_dim)

    def forward(self, x):
        x = self.patch_embedding(x)  # [batch_size, hidden_dim, 14, 14]
        x = x + self.pos_embedding
        x = x.flatten(2).permute(2, 0, 1)  # [14*14, batch_size, hidden_dim]
        x = self.transformer_encoder(x)  # [14*14, batch_size, hidden_dim]
        x = x.permute(1, 0, 2).flatten(1, 2)  # [batch_size, 14*14*hidden_dim]
        x = self.fc(x)
        return x

input_dim = 28*28
hidden_dim = 64
output_dim = 10
num_layers = 2
model_tr = TransformerEncoderModel(input_dim, hidden_dim, output_dim, num_layers)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
def fit_one_epoch(train_loader, model, optimizer, criterion, log_to_wandb):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(train_loader, desc="Training Batches"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100.0 * correct / total
    if log_to_wandb:
        wandb.log({"train_loss": train_loss, "train_accuracy": train_accuracy})
    print(f'Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')

def test(model, test_loader, log_to_wandb):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Testing Batches"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_accuracy = 100.0 * correct / total
    if log_to_wandb:
        wandb.log({"test_accuracy": test_accuracy})
    return test_accuracy


In [ ]:
def main(model, train_sizes, log_to_wandb, name):
    accuracies = []
    batch_size = 64
    for train_size in train_sizes:
        accs = []
        # name = "trainsize_{}".format(train_size)
        name = name+'_{}'.format(train_size)
        if log_to_wandb:
            run = wandb.init(
                name = name,
                id = name,
                project="CV-Assignment-2",
                config={
                    "learning_rate": 0.01,
                    "epochs": 10,
                    "batch_size":batch_size
                },
            )

        model = model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        train_subset, _ = torch.utils.data.random_split(train_dataset, [train_size, len(train_dataset) - train_size])
        train_loader_subset = torch.utils.data.DataLoader(dataset=train_subset,
                                                           batch_size=batch_size,
                                                           shuffle=True)

        for epoch in range(num_epochs):
            print(f"Epoch {epoch + 1}/{num_epochs}")
            fit_one_epoch(train_loader_subset, model, optimizer, criterion, log_to_wandb)
            test_accuracy = test(model, test_loader, log_to_wandb)
            print(f'Test Accuracy for {epoch}: {test_accuracy:.2f}%')
            accs.append(test_accuracy)
        accuracies.append(accs[-1])
    return accuracies

# Experiment 1: Varying the number of training samples with CNN


# # Experiment 2: Varying the number of training samples with Transformer Encoder
# print("\nTransformer Encoder Model:")
# train_sizes = [6000, 60000]
# accuracies = train(TransformerEncoder(1, 64, 10, 2), train_sizes)

# Plot the results
# plt.figure(figsize=(10, 6))
# plt.plot(train_sizes, accuracies, marker='o')
# plt.xlabel('Number of training samples')
# plt.ylabel('Accuracy (%)')
# plt.title('Classification accuracy vs. number of training samples')
# plt.show()

In [ ]:
print("CNN Model:")
train_sizes = [600, 1800, 6000, 18000, 60000]
log_to_wandb = False
name = 'model_tr'
accuracies = main(model_tr, train_sizes, log_to_wandb, name)